In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-12-05 23:20:34.957077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

In [3]:
x_train = np.load("./seq_data/x_train.npy")
y_train = np.load("./seq_data/y_train.npy")
x_val = np.load("./seq_data/x_val.npy")
y_val= np.load("./seq_data/y_val.npy")

In [4]:
MAX_PLAY_LENGTH = 203

In [5]:
import keras.backend as K

def my_loss(y_true, y_pred):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    #tf.print(y_true)
    true = K.reshape(y_true, (-1, y_true.shape[-1]))
    #tf.print(true)
    pred = K.reshape(y_pred, (-1, y_pred.shape[-1]))
    return bce(true[:,0:-1], pred[:,0:-1]) + mse(true[:,-1], pred[:,-1])

def bce_metric(y_true, y_pred):
    true = K.reshape(y_true, (-1, y_true.shape[-1]))
    pred = K.reshape(y_pred, (-1, y_pred.shape[-1]))
    return K.mean(K.binary_crossentropy(true[:,0:-1], pred[:,0:-1], from_logits=False))

def mse_metric(y_true, y_pred):
    true = K.reshape(y_true, (-1, y_true.shape[-1]))
    pred = K.reshape(y_pred, (-1, y_pred.shape[-1]))
    return K.mean(K.square(true[:,-1] - pred[:,-1]), axis=-1)

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_pred):
    true = K.reshape(y_true, (-1, y_true.shape[-1]))
    pred = K.reshape(y_pred, (-1, y_pred.shape[-1]))
    true_positives = K.sum(K.round(K.clip(true[:,1] * pred[:,1], 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true = K.reshape(y_true, (-1, y_true.shape[-1]))
    pred = K.reshape(y_pred, (-1, y_pred.shape[-1]))
    true_positives = K.sum(K.round(K.clip(true[:,1] * pred[:,1], 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(pred[:,1], 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [6]:
''' 

input = (m, max_length, 23*7)
outut = (m, max_length, 3)      - at every time step, predict sack probability and time to sack

y = (m, max_length, 3)

'''

def createModel(input_shape = (203, 23*7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    lstm = tfl.LSTM(100, activation='tanh', recurrent_activation='tanh', return_sequences=True)(X)
    drop = tfl.Dropout(0.2)(lstm)
    d3 = tfl.Dense(3,activation=None)(drop)
    permute = tfl.Permute((2,1))(d3)    # change input from (None, 203, 3) to (None, 3, 203)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    probs = tfl.Cropping1D(cropping=(0,1))(permute) # shape (None, 2, 203)
    probs = tfl.Softmax(axis=1)(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(permute) # shape (None, 1, 203)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=1)([probs, time]) # shape (None, 3, 203)
    out = tfl.Permute((2,1))(out) # shape (None, 203, 3)
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model


In [7]:
model = createModel()

print(model.summary())

2022-12-05 23:21:08.100698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 203, 161)]   0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 203, 100)     104800      ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 203, 100)     0           ['lstm[0][0]']                   
                                                                                                  
 dense (Dense)                  (None, 203, 3)       303         ['dropout[0][0]']                
                                                                                              

In [8]:
LEARNING_RATE = 0.001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS,
    clipvalue=0.1)

model.compile(loss = my_loss, optimizer = opt, metrics = [metrics.CategoricalAccuracy(), bce_metric, mse_metric, recall, precision])

In [9]:
x_train_input = x_train.reshape(-1, MAX_PLAY_LENGTH, 23, 11)[:,:,:,4:].reshape(-1,MAX_PLAY_LENGTH,23*7)

print(f"input (X) shape = {x_train_input.shape}")
print(f"y shape = {y_train.shape}")

input (X) shape = (6000, 203, 161)
y shape = (6000, 203, 3)


In [11]:
NUM_EPOCHS = 2
history = model.fit(x_train_input, y_train, epochs=NUM_EPOCHS, batch_size=1)

KeyboardInterrupt: 

In [12]:
x_train_input.shape

(60, 203, 161)

In [21]:
# confirm every play has some non-zero elements
np.all(x_train_input.reshape(x_train_input.shape[0],-1), axis=-1)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [17]:
np.all([0,2,3])

False